In [ ]:
# --- 1. Install Dependencies ---
!pip install fastapi uvicorn pyngrok transformers torch

# --- 2. Authenticate ngrok ---
# Get your token from: https://dashboard.ngrok.com/get-started/your-authtoken
from pyngrok import ngrok
NGROK_TOKEN = "ngrok token"  # <--- REPLACE THIS
ngrok.set_auth_token(NGROK_TOKEN)

In [2]:
%%writefile server.py

# --- 3. Write Server Code to File ---
# We write the code to a file named 'server.py' so uvicorn can run it

import torch
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel, Field, validator
from transformers import AutoTokenizer, AutoModel
import torch.nn.functional as F

app = FastAPI()

# Load Model (Global for warm start)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Loading ESM2-650M to {device}...")
model_name = "facebook/esm2_t33_650M_UR50D"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name).to(device).eval()

class MutationRequest(BaseModel):
    sequence: str
    mutation_code: str

@app.post("/extract")
async def extract(req: MutationRequest):
    try:
        # 1. Parse Mutation (e.g. A23V)
        wt_aa = req.mutation_code[0]
        mut_aa = req.mutation_code[-1]
        pos = int(req.mutation_code[1:-1]) - 1 # 0-indexed

        # 2. Validation
        if req.sequence[pos] != wt_aa:
            raise HTTPException(400, f"WT mismatch: {req.sequence[pos]} at {pos+1} != {wt_aa}")

        # 3. Create Mutant Sequence
        seq_list = list(req.sequence)
        seq_list[pos] = mut_aa
        mut_seq = "".join(seq_list)

        # 4. Extraction Helper
        def get_emb(s, p):
            inputs = tokenizer(s, return_tensors="pt", add_special_tokens=True).to(device)
            with torch.no_grad():
                out = model(**inputs)
            # ESM adds <cls> at 0, so target is p+1
            return out.last_hidden_state[0, p + 1, :]

        # 5. Get Embeddings
        wt_vec = get_emb(req.sequence, pos)
        mut_vec = get_emb(mut_seq, pos)

        # 6. Compute Features
        delta = mut_vec - wt_vec
        abs_delta = torch.abs(delta)
        l2 = torch.dist(wt_vec, mut_vec, 2).item()
        cos = F.cosine_similarity(wt_vec.unsqueeze(0), mut_vec.unsqueeze(0)).item()

        return {
            "wt_embedding": wt_vec.tolist(),
            "mut_embedding": mut_vec.tolist(),
            "delta_embedding": delta.tolist(),
            "abs_delta_embedding": abs_delta.tolist(),
            "cosine_similarity": cos,
            "l2_distance": l2
        }
    except Exception as e:
        raise HTTPException(500, str(e))

Writing server.py


In [3]:
import os
from pyngrok import ngrok

# Open a tunnel to port 8000
# This generates the PUBLIC URL you need
public_url = ngrok.connect(8000).public_url
print(f"✅ YOUR SERVER URL IS:  {public_url}  ")
print(f"👉 Copy this URL into your Client App")

# Run Uvicorn in the background
get_ipython().system_raw('uvicorn server:app --reload --port 8000 &')

✅ YOUR SERVER URL IS:  https://miracle-subparietal-liv.ngrok-free.dev  
👉 Copy this URL into your Client App
